# Вычисление нормированной энтропии Шеннона по столбцам MSA

В этом ноутбуке:
1. Считаем множественное выравнивание (MSA) из файла `../data/processed/06_seq_for_alphafold/msa_all_clades_representatives.fasta`.
2. Для каждого столбца (позиции выравнивания) вычислим энтропию Шеннона, игнорируя гэпы ('-') и неизвестные ('X').
3. Нормируем энтропию, деля её на log2(20) (максимальная энтропия для 20 аминокислот).
4. Для каждой из 39 последовательностей формируем TSV-файл:
   `Position_in_protein\tNormalized_Entropy`, где `Position_in_protein` — счётчик по аминокислотам 
   (без учёта гэпов в MSA), а `Normalized_Entropy` — энтропия позиции (колонки) MSA, нормированная в [0,1].

In [1]:
import math
from pathlib import Path
from Bio import AlignIO

In [2]:
msa_path = Path("../data/processed/06_seq_for_alphafold/msa_all_clades_representatives.fasta")
output_dir = Path("../data/processed/07_shannon_entropy/")
output_dir.mkdir(parents=True, exist_ok=True)

In [3]:
# Читаем MSA (формат FASTA)
alignment = AlignIO.read(str(msa_path), "fasta")
n_sequences = len(alignment)
aln_length = alignment.get_alignment_length()
print(f"Прочитано {n_sequences} последовательностей, длина выравнивания = {aln_length}")


Прочитано 39 последовательностей, длина выравнивания = 2524


In [4]:
def shannon_entropy(column):
    """
    column: строка символов длины n_sequences (по одной позиции выравнивания)
    Возвращает энтропию Шеннона, игнорируя символы '-' и 'X'.
    """
    freqs = {}
    total = 0
    for aa in column:
        if aa == '-' or aa.upper() == 'X':
            continue
        aa = aa.upper()
        freqs[aa] = freqs.get(aa, 0) + 1
        total += 1

    if total == 0:
        return 0.0  # если во всех строках gap

    entropy = 0.0
    for count in freqs.values():
        p = count / total
        entropy -= p * math.log2(p)
    return entropy

In [5]:
max_entropy = math.log2(20)  # максимальная энтропия при 20 аминокислотах ≈ 4.322
entropies = [0.0] * aln_length
normalized_entropies = [0.0] * aln_length

for i in range(aln_length):
    column = alignment[:, i]
    h = shannon_entropy(column)
    entropies[i] = h
    normalized_entropies[i] = h / max_entropy

print("Первые 10 ненормированных энтропий:", entropies[:10])
print("Первые 10 нормированных энтропий:", normalized_entropies[:10])

Первые 10 ненормированных энтропий: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Первые 10 нормированных энтропий: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [6]:
for record in alignment:
    seq_id = record.id
    seq = str(record.seq)
    # Создаём "безопасное" имя файла
    safe_id = "".join(c if (c.isalnum() or c in "-_") else "_" for c in seq_id)
    out_path = output_dir / f"conservation_normalized_{safe_id}.tsv"

    with open(out_path, "w") as fout:
        fout.write("Position_in_protein\tNormalized_Entropy\n")
        pos_counter = 0
        for i, aa in enumerate(seq):
            if aa == '-' or aa.upper() == 'X':
                continue
            pos_counter += 1
            fout.write(f"{pos_counter}\t{normalized_entropies[i]:.5f}\n")
    print(f"  → Сохранён файл {out_path.name} ({pos_counter} позиций)")

  → Сохранён файл conservation_normalized_WP_249755640_1.tsv (689 позиций)
  → Сохранён файл conservation_normalized_WP_159144195_1.tsv (353 позиций)
  → Сохранён файл conservation_normalized_WP_130970024_1.tsv (380 позиций)
  → Сохранён файл conservation_normalized_CP003678_1_2534.tsv (412 позиций)
  → Сохранён файл conservation_normalized_OZ007026_1_1757.tsv (363 позиций)
  → Сохранён файл conservation_normalized_NZ_CP120635_1_1445.tsv (209 позиций)
  → Сохранён файл conservation_normalized_NZ_LT634361_1_332.tsv (691 позиций)
  → Сохранён файл conservation_normalized_NZ_CP151867_1_1971.tsv (483 позиций)
  → Сохранён файл conservation_normalized_CP034683_1_1272.tsv (545 позиций)
  → Сохранён файл conservation_normalized_CP045340_1_216.tsv (507 позиций)
  → Сохранён файл conservation_normalized_OY282455_1_1419.tsv (429 позиций)
  → Сохранён файл conservation_normalized_NZ_CP069280_1_2106.tsv (317 позиций)
  → Сохранён файл conservation_normalized_NZ_CP147916_1_1085.tsv (535 позиций)
  